In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data augmentation

# Define image dimensions (adjust based on your image size)
IMAGE_HEIGHT = 224  # Example height
IMAGE_WIDTH = 224  # Example width

def load_data(data_dir):
  images, labels = [], []
  for filename in os.listdir(data_dir):
    # Check if it's an image file
    if filename.endswith(('.jpg', '.png', '.jpeg')):
      image_path = os.path.join(data_dir, filename)
      image = cv2.imread(image_path)
      image = cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH))
      image = image.astype('float32') / 255.0  # Normalize pixel values

      # Find the corresponding label file (assuming same filename without extension)
      label_path = os.path.join(data_dir, os.path.splitext(filename)[0] + ".txt")
      if not os.path.exists(label_path):
        raise ValueError(f"Label file not found for image: {filename}")

      # Parse the first line of the label file (assuming format: class_id)
      with open(label_path, 'r') as f:
        label_data = f.readline()
      label = int(label_data.split()[0])  # Assuming class ID is the first element

      images.append(image)
      labels.append(label)  # 0 for crop, 1 for weed (adjust based on your labeling scheme)
  return np.array(images), np.array(labels)

# Load your entire dataset
data_dir = 'C:/Users/Pratham/Downloads/data'  # Replace with your data directory path
all_images, all_labels = load_data(data_dir)

# Split the data into training and validation sets (common practice: 80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

# Data augmentation (optional but highly recommended)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = datagen.flow(X_val, y_val, batch_size=32)

# Define the CNN model for binary classification
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)






Epoch 1/10


33/33 [==============================] - 40s 1s/step - loss: 1.4842 - accuracy: 0.5740 - val_loss: 0.6472 - val_accuracy: 0.6692
Epoch 2/10
33/33 [==============================] - 39s 1s/step - loss: 0.5795 - accuracy: 0.7000 - val_loss: 0.4678 - val_accuracy: 0.7731
Epoch 3/10
33/33 [==============================] - 37s 1s/step - loss: 0.5028 - accuracy: 0.7683 - val_loss: 0.4661 - val_accuracy: 0.8077
Epoch 4/10
33/33 [==============================] - 36s 1s/step - loss: 0.4288 - accuracy: 0.8269 - val_loss: 0.4181 - val_accuracy: 0.8385
Epoch 5/10
33/33 [==============================] - 36s 1s/step - loss: 0.3882 - accuracy: 0.8567 - val_loss: 0.3454 - val_accuracy: 0.8577
Epoch 6/10
33/33 [==============================] - 39s 1s/step - loss: 0.3638 - accuracy: 0.8606 - val_loss: 0.3496 - val_accuracy: 0.8615
Epoch 7/10
33/33 [==============================] - 50s 1s/step - loss: 0.3192 - accuracy: 0.8769 - val_loss: 0.3605 - val_accuracy: 0.8654
Epoch 8/10
33/33

In [4]:
loss, accuracy = model.evaluate(val_generator)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy*100 ,'%')

9/9 [==============================] - 6s 671ms/step - loss: 0.2851 - accuracy: 0.9038
Validation Loss: 0.2850640118122101
Validation Accuracy: 90.38461446762085 %
